# 제주 한달살기 숙소 추천을 위한 분석 - 크롤링


### 미스터멘션 - https://www.mrmention.co.kr/
### 한달살기 전문사이트
- 이유 : 한달살기를 위한 시설이 준비된 숙소
- case 155건

## 미스터멘션 
메뉴 - 한달살이 best <br>
지역 검색 - 제주 <br>
https://www.mrmention.co.kr/v2/search?theme=31&title=%EC%A0%9C%EC%A3%BC&as_path=0

In [ ]:
# 가상환경 - project1
# !pip install selenium pandas bs4 xlrd requests tqdm pyautogui

In [ ]:
import pandas as pd
import numpy as np

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup
import requests


from urllib.parse import quote

import time
#time.sleep을 위한 난수생성
import random

from tqdm import tqdm

# text 전처리
import re

# 마우스 커서 이동을 위한 패키지
import pyautogui

### 크롤링 완성 코드

In [ ]:
url = 'https://www.mrmention.co.kr/v2/search?theme=31&title=%EC%A0%9C%EC%A3%BC&as_path=0'

driver = webdriver.Chrome()
driver.get(url)

# 화면이 뜰때까지 기다리기
driver.implicitly_wait(10)
# 창 최대화
driver.maximize_window()
time.sleep(2.4)

# 지도창 닫기
driver.find_element(By.CLASS_NAME, 'search_map_toggle_btn__GAhJd').click()

### 스크롤 내리기 위한 영역 선택이 어려움
# 이를 해결하기 위해 마우스 커서를 해당 영역에 두고, Page_down 실행으로 스크롤 내리기

# 마우스 이동
pyautogui.moveTo(465 , 800 , 2)	
pyautogui.leftClick()

# 스크롤을 내려서 모든 숙소 확인
for _ in range(30):
    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)  
    time.sleep(round(random.uniform(0.3, 1.2), 3))
time.sleep(round(random.uniform(1.3, 2.2), 3))

### 기본 정보 크롤링
# 첫 페이지에서 크롤링
# 이름 / 숙박가능기간 / 가격 / 평점 / 리뷰수 / 지역
soup = BeautifulSoup(driver.page_source, 'html.parser')
accom_list =soup.select('.jsx-5f908135cc97e6c3 > .AccommodationCard_room_info_con__hyAro')

temp = []
for accom in accom_list:
    title = accom.select_one('.AccommodationCard_room_tit__f6lnu').text.strip()
    day = accom.select_one('.AccommodationCard_amount__dyci8').select('.Price_day__SUxE_')[-1].text.strip()
    price = accom.select_one('.AccommodationCard_amount__dyci8').select('.Price_price__mdIcc')[-1].text.strip()
    price = re.sub('[^0-9]','',price)
    rating = accom.select_one('.AccommodationCard_rating_sco__75Yd9').text.strip()
    reviewNum = accom.select_one('.AccommodationCard_rating_count__fesL7').text.strip()[1:-1]
    location = accom.select_one('.AccommodationCard_area___0RDd').text.strip()

    temp.append({'title' : title,
                 'day' : day,
                 'price' : price,
                 'rating' : rating,
                 'reviewNum' : reviewNum,
                 'location' : location})
    
temp = pd.DataFrame(temp)
temp.to_csv('./data/mmention_20240118.csv', index=False)

### 상세정보
# 첫화면 soup한 정보에서 개별 url 링크 확인
# 개별적으로 들어가서 상세정보 확인

pension_list = soup.select_one('.search_search_list__Zfhwh').select('a')
review_result = []
detail_result = []

for pension in pension_list:
    sub_url = pension.attrs['href']
    base_url = 'https://www.mrmention.co.kr/'
    url = base_url + sub_url
    driver.get(url)

    driver.maximize_window()
    driver.implicitly_wait(10)    
    
    ### 구비시설 
    driver.find_element(By.CLASS_NAME, 'Amenity_ame_more__rOwp1').click()
    time.sleep(round(random.uniform(1.0, 1.5), 3))

    ammen_list = driver.find_element(By.CLASS_NAME, 'AmenityPopup_ame_pop_con__NhQad').find_elements(By.TAG_NAME, 'ul')
    ammen_result = []
    for ammen in ammen_list:
        ammen_result.append(ammen.text.strip())
    
    
    driver.find_element(By.CLASS_NAME, 'AmenityPopup_close__8jpjy').click()
    time.sleep(round(random.uniform(1.0, 1.5), 3))

    ### review
    # 스크롤을 끝까지 내려서 모든 정보 획득
    # 크롤링 방지를 피하기 위해 한번에 스크롤 내리지 않고, 속임수를 사용(time.sleep 을 random 시간으로)
    while True:
        last_height = driver.execute_script("return document.body.scrollHeight")

        # 창의 내리기 끝까지 하지않고, 중간에 멈춰서 한번은 page_down을 통해서
        driver.execute_script(f'window.scrollTo(0, {last_height - random.randint(1, 100)})')
        time.sleep(round(random.uniform(1.0, 1.5), 3))
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)     
        time.sleep(round(random.uniform(0.3, 1.2), 3))

        # 창의 높이가 달라지지 않으면 스크롤 내리기 중단
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break

    # 리뷰 더보기 버튼 클릭
    driver.find_elements(By.CLASS_NAME, 'ToggleBtn_accom_toggle_btn_blur__pg8Mg')[1].click()
    time.sleep(round(random.uniform(1.3, 2.2), 3))

    mini_soup = BeautifulSoup(driver.page_source, 'html.parser')

    title = mini_soup.select_one('.AccommodationTitle_title__kNF2R').text
    review_list = mini_soup.select_one('#Review_review__5RC6b').select('li')
    
    #리뷰가 없는 경우 pass
    try :
        for review in review_list:
            review_text = review.select_one('.Review_is_review__txxuK').text
            review_date = review.select_one('.Review_date__yjft1').text
            review_star = review.select_one('.Star_frontStarDiv__xU1lg').get_attribute_list('style')[0]
            review_star = re.sub('[^0-9]', '',review_star)
            review_star = int(review_star) / 20
            review_result.append([title, review_text, review_date, review_star, ammen_result])
    except :
        review_text = 'none'
        review_date = 'none'
        review_star = 'none'
        review_result.append([title, review_text, review_date, review_star, ammen_result, sub_url])
        

review_result = pd.DataFrame(review_result)
review_result.columns = ['title', 'review', 'review_date', 'star', 'ammen', 'sub_url']
review_result.to_csv('./data/mmention_review_20240125.csv', index=False)